In [1]:
import numpy as np
import scipy.stats as st

# Function returns the index of the value in an array nearest to the input value
def find_nearest(vector, value):
    idx = (np.abs(vector-value)).argmin()
    return idx

# Function signalFloor to iteratively find the appropriate signal floor for 
# a given background level, at a given confidence level. This calculation
# assumes background-only and solves the cumulative Poisson distribution iteratively.

def signalFloor(Bg, conf, accuracy=1e-6):

    #print(len(Bg))
    floor = np.empty_like(Bg, dtype=float)
    
    # Convert the confidence level to a cdf:
    conf = 1-conf
        
    for i in range(0, len(Bg)):
            
        #print(i)
        
        
        # First, calculate the expectation with a background-only hypothesis
        floor_bg = Bg[i] - st.poisson.ppf(conf, Bg[i])

        # Make an interval around floor_bg
        interval = np.linspace(0.5*floor_bg, max(2*floor_bg,6), 1000)

        # Reduce the interval iteratively until reaching desired
        deviations = ( conf - st.poisson.cdf(round(Bg[i]), Bg[i]+interval) )


        # Now, iteratively reduce the interval until the desired decimal precision is reached
        while np.max(interval)-np.min(interval) > accuracy:
  
            upper_lim = interval[np.all( [ (deviations > 0), (deviations <= np.min(deviations[deviations > 0])) ], axis=0 )]

            lower_lim = interval[np.all( [ (deviations < 0), (deviations >= np.max(deviations[deviations < 0])) ], axis=0 )]

            interval = np.linspace(lower_lim, upper_lim, 1000)
            deviations = ( conf - st.poisson.cdf(round(Bg[i]), Bg[i]+interval) )

            flr = np.mean(interval)
            
            floor[i] = flr
    
    return floor


In [2]:
signalFloor(Bg=[0.0], conf=0.90)[0]

2.302585092825784

In [3]:
signalFloor(Bg=[0.0], conf=0.95)[0]

2.9957322736197707

In [4]:
signalFloor(Bg=[4.5], conf=0.9)[0]

3.4935895871111624